<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Tomás Fontecilla </em><br>

</div>

# Machine Learning Avanzado
*02 de Diciembre de 2024*

#### Integrantes: 
` Gabriel Álvarez - Rosario Valderrama `

## 1. Objetivo

El objetivo de este informe es ajustar **redes neuronales Feed-Forward (FNN)** y **redes neuronales convolucionales (CNN)** para crear modelos que puedan predecir si cierta imagen corresponde a un chihuahua o a un muffin utilizando la base extraída de Kaggle ` chihuahuas vs muffins `. 

La metodología a utilizar es que se crearán, entrenarán y evaluarán modelos **FNN** con diferentes estructuras y luego se realizará el mismo proceso pero agregando capas convolucionales para crear, convirtiendo los modelos en **CNN**. Esto es realizado para poder comparar los resultados obtenidos y determinar cuál de los modelos es el más adecuado para el problema planteado.

Los modelos tendrán combinaciones de estructuras, agregando o quitando Regularización L2, Dropout y variando la tasa de aprendizaje:


In [7]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, Sequential, regularizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay

## 2. Introducción

El dataset ` chihuahuas vs muffins ` contiene 5.917 imágenes divididas en entrenamiento y testeo de chihuahuas y muffins, donde se busca entrenar modelos de redes neuronales FNN y CNN para lograr clasificar correctamente cada imagen. Para esto, los pasos a seguir son:

1. Cargar las imágenes y etiquetas
2. Preprocesar las imágenes si es necesario
3. Crear el modelo de red neuronal
4. Entrenar el modelo de red neuronal
5. Evaluar el modelo de red neuronal

## 3. Carga de dataset

Para comenzar, se carga las imágenes desde las carpetas data/train y data/test, donde se infieren las etiquetas de ellas basándose en los nombres de las subcarpetas. Todas las imágenes se redimensionan a 128 x 128 para asegurar que todas tengan el mismo tamaño, para luego agruparlas en lotes de 32 imágenes, lo que permite poder procesarlas de forma más eficiente. Todas las imágenes son "barajadas" del conjunto de entrenamiento, para así evitar patrones en el orden de los datos que finalmente afecte este entrenamiento.

Se normalizan los valores de cada pixel de las imágenes, dividiéndolos por 255, para que así estén dentro del rango de 0 a 1. Si no se aplicara esta normalización, los valores podrían estar en el rango del 0 a 255, dificultando así la convergencia del modelo.

In [8]:
# Directorios
train_dir = "data/train"
test_dir = "data/test"

# Carga de datos de entrenamiento
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=True
)

# Carga de datos de testeo
test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)

# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

# Normalización (valores entre 0 y 1)
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


Dado a que el perceptrón multicapa no puede procesar las imágenes en 2D, requiere que las imágenes se transformen en vectores en una dimensión. Para esto, se busca aplanar las imágenes, convirtiendo cada imagen de 128 x 128 x 3 (RGB) en un vector 1D de tamaño 128\*128\*3 = 49152.

In [9]:
# Aplanar las imágenes para el MLP
def flatten_images(x, y):
    flat_dim = 128 * 128 * 3 
    x = tf.reshape(x, [tf.shape(x)[0], flat_dim])
    return x, y

train_dataset_flat = train_dataset.map(flatten_images)
test_dataset_flat = test_dataset.map(flatten_images)


## 4. Creación y evaluación de modelos
### 4.1 Redes Neuronales Feed-Forward (FNN)
#### 4.1.1 Modelo 1

El modelo del FNN se construye utilizando la API **Sequiential** de TensorFlow/Keras, lo que permite definir una arquitectura de red neuronal de manera secuencial, que va atravesando capa por capa.

La primera capa (Input) especifica la forma de las entradas del modelo, que corresponde a vectores del tamaño 128 x 128 x 3 = 49152, resultado de aplanar las imágenes RGB de entrada que previamente se redimencionaron a 128 x 128 pixeles. 

Posteriormente, se definen 3 capas ocultas densas llamadas **Dense**, que están completamente conectadas. Esto significa que cada neurona de una capa está conectada a todas las neuronas de la siguiente capa. La primera capa oculta contiene 256 neuronas, la segunda 128 neuronas y la tercera 64 neuronas.

Todas las capas ocultas usan la función de activación ReLU, la que se selecciona por su capacidad para introducir no linealidad en la red, permitiendo que el modelo aprenda relaciones complejas entre los datos de entrada y los de salida. Esta función activa únicamente valores positivos (los deja sin cambios) y transforma los valores negativos a cero. Esto mejora la eficiencia computacional en comparación con otras funciones de activación y ayuda a evitar problemas como el desvanecimiento del gradiente, que puede ralentizar el aprendizaje en redes profundas. Al activar solo ciertas neuronas según las entradas, ReLU también ayuda a que la red se enfoque en características relevantes, optimizando su capacidad de generalización.

Finalmente, la capa de salida también es densa y tiene tantas neuronas como clases en el problema, lo cual se especifica como len(class_names). En este caso, las clases corresponden a las categorías de las imágenes (chihuahuas y muffins). Esta capa utiliza la función de activación softmax, que convierte las salidas en probabilidades normalizadas, asignando una probabilidad a cada clase. Se utiliza esta activación ya que permite interpretar la salida del modelo como la probabilidad de que una imagen pertenezca a cada clase.

En conjunto, estas capas y funciones de activación permiten que el modelo procese los datos de entrada, aprenda patrones significativos en las imágenes y genere predicciones probabilísticas sobre las clases a las que pertenece cada imagen. 

In [10]:
# Crear el modelo MLP
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation='softmax')
])

**mlp_model.compile** configura el modelo de Perceptrón Multicapa para el entrenamiento, definiendo tres componentes clave: el optimizador, la función de pérdida y las métricas de evaluación. El optimizador Adam se utiliza por su capacidad para ajustar dinámicamente la tasa de aprendizaje, combinando las ventajas de métodos como Momentum y RMSProp. La función de pérdida sparse_categorical_crossentropy es adecuada para problemas de clasificación multiclase con etiquetas enteras, como las clases inferidas de las imágenes. Finalmente, se especifica la métrica de exactitud (accuracy), que mide la proporción de predicciones correctas durante el entrenamiento y la evaluación, proporcionando una evaluación clara del desempeño del modelo.

In [11]:
# Compilar el modelo
mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

**mlp_model.fit** se utiliza para entrenar el modelo con los datos del conjunto de entrenamiento (train_dataset_flat). La cantidad de "epochs" se define mediante el parámetro epochs=10, indicando que el modelo realizará 10 iteraciones completas sobre todo el conjunto de datos de entrenamiento. Durante cada "epoch", el modelo ajusta sus pesos en función de los gradientes calculados para minimizar la función de pérdida. Cada pasada completa permite al modelo mejorar su capacidad para aprender los patrones en los datos, mientras que un número excesivo de "epochs" podría llevar al sobreajuste, donde el modelo se adapta demasiado a los datos de entrenamiento y pierde capacidad de generalización para datos nuevos. 

In [12]:
# Entrenar el modelo
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 125ms/step - accuracy: 0.4916 - loss: 8.0189
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 122ms/step - accuracy: 0.5544 - loss: 0.6746
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 124ms/step - accuracy: 0.5381 - loss: 0.6910
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 124ms/step - accuracy: 0.5415 - loss: 0.6888
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 123ms/step - accuracy: 0.5403 - loss: 0.6898
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 123ms/step - accuracy: 0.5412 - loss: 0.6909
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 124ms/step - accuracy: 0.5416 - loss: 0.6896
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 123ms/step - accuracy: 0.5393 - loss: 0.6878
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 123ms/step - accuracy: 0.5415 - loss: 0.6899
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 124ms/step - accuracy: 0.5419 - loss: 0.6870


El modelo muestra una disminución de la pérdida y un aumento de la exactitud en la primera epoch, pero queda estancado desde la segunda en adelante. Esto indica que el modelo empezó a aprender, pero luego no mejoró significativamente su desempeño.

**mlp_model.evaluate** permite evaluar el desempeño del modelo en el conjunto de datos de prueba (test_dataset_flat). Este proceso calcula la pérdida, que mide qué tan bien o mal se comporta el modelo con datos nuevos, y el accuracy, que indica el porcentaje de predicciones correctas. 

La pérdida ayuda a diagnosticar posibles problemas en el ajuste del modelo, mientras que el accuracy refleja su capacidad de generalización y su desempeño al momento de clasificar las imágenes de salida. 

In [13]:
# Evaluar el modelo
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida Modelo 1 FNN: {loss}")
print(f"Exactitud Modelo 1 FNN: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.8584 - loss: 0.6422
Pérdida Modelo 1 FNN: 0.6899157166481018
Exactitud Modelo 1 FNN: 0.5405405163764954


Se puede observar que el accuracy final es de 0.54, valor cercano al 50% que se obtendría al azar, lo que indica que el modelo no logra clasificar correctamente las imágenes de chihuahuas y muffins.

#### 4.1.2 Modelo 2: Regularización L2 y Dropout

Se agrega regularización L2 a las capas ocultas del modelo, lo que permite penalizar los pesos de la red para evitar el sobreajuste.

In [14]:
# Crear el modelo MLP con regularización
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(len(class_names), activation='softmax')
])

# Compilar el modelo
mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Entrenar el modelo
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 25s 160ms/step - accuracy: 0.5036 - loss: 10.8483
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5540 - loss: 2.4760
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.5128 - loss: 2.0551
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5107 - loss: 1.8892
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.5085 - loss: 1.8035
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5182 - loss: 1.6484
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.5044 - loss: 1.5463
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.4920 - loss: 1.4472
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5060 - loss: 1.3625
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5148 - loss: 1.2880


El modelo muestra una disminución constante de la pérdida a través de las epochs, aunque el accuracy se mantiene en un nivel bajo. Esto sugiere que el modelo está aprendiendo a clasificar las imágenes, pero no logra generalizar adecuadamente para el conjunto de datos de entrenamiento.

In [16]:
# Evaluar el modelo
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida Modelo 2 FNN: {loss}")
print(f"Exactitud Modelo 2 FNN: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.1416 - loss: 1.3985
Pérdida Modelo 2 FNN: 1.299899697303772
Exactitud Modelo 2 FNN: 0.45945945382118225


En comparación con el primer modelo, el agregar regulación L2 hizo empeorar los resultados debido al aumento en el valor de pérdida y a la disminución en el accuracy a un 45%, incluso aún más bajo que el 50% que se esperaría obtener al azar. Esto indica que la regularización L2 no logra mejorar el desempeño del modelo.

#### 4.1.3 Modelo 3: Regularización L2, Dropout y tasa de aprendizaje

Para prevenir el sobreajuste en este modelo, se recomienda utilizar una combinación de tres técnicas de regularización: Dropout, Regularización L2, y Decay. 

**Dropout** introduce robustez al modelo al apagar aleatoriamente un porcentaje de las neuronas durante el entrenamiento, obligando al modelo a aprender patrones más generalizados en lugar de depender de combinaciones específicas de pesos. Esto se implementa añadiendo capas de Dropout con un porcentaje, como layers.Dropout(0.3) para desactivar el 30% de las neuronas en cada capa. 

**Regularización L2**, penaliza los pesos grandes al añadir una restricción basada en la suma de los cuadrados de los pesos, lo que fomenta que el modelo mantenga distribuciones de pesos más balanceadas. Se configura con el argumento kernel_regularizer=regularizers.l2(0.01) en las capas densas.

**Decay** ajusta dinámicamente la tasa de aprendizaje del optimizador, permitiendo pasos grandes al inicio para aprender rápidamente y pasos más pequeños al final para refinar los pesos; esto se implementa mediante un programa de decaimiento como ExponentialDecay. 

Estas técnicas ayudan a mejorar la generalización del modelo al reducir la complejidad, distribuir el aprendizaje de manera uniforme y evitar la dependencia excesiva en subconjuntos específicos de parámetros.

In [17]:
# Decaimiento de la tasa de aprendizaje
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=False
)

# Crear el modelo con Dropout y Regularización L2
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Compilar el modelo con Decay
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

mlp_model.compile(
    optimizer=optimizer,  # Optimización con Decay
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 25s 161ms/step - accuracy: 0.5032 - loss: 66.7904
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.5113 - loss: 2.2613
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5322 - loss: 1.5939
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5288 - loss: 1.4917
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5377 - loss: 1.2985
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 161ms/step - accuracy: 0.5385 - loss: 1.1888
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.5402 - loss: 1.2049
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5299 - loss: 1.0549
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.5370 - loss: 0.9967
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.5406 - loss: 0.9606


El modelo muestra una disminución constante de la pérdida a través de las epochs, aunque el accuracy se mantiene en un nivel bajo. Esto sugiere que el modelo está aprendiendo a clasificar las imágenes, pero no logra generalizar adecuadamente para el conjunto de datos de entrenamiento.

In [19]:
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida Modelo 3 FNN: {loss}")
print(f"Exactitud Modelo 3 FNN: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8584 - loss: 0.9061
Pérdida Modelo 3 FNN: 0.9332302212715149
Exactitud Modelo 3 FNN: 0.5405405163764954


Al igual que en ambos modelos anteriores, el accuracy obtenido al evaluar el modelo es cercano al 50%, lo que indica que el modelo no logra clasificar correctamente las imágenes de chihuahuas y muffins.

### 4.2 Redes Neuronales Convolucionales (CNN)

#### 4.2.1 Modelo 1: Dropout

A continuación se presentan tres modelos de redes neuronales convolucionales, cada uno con una estructura diferente. Su estructura varía en la combinación de tres técnicas de regularización: Dropout, Regularización L2, y Decay.

Cabe mencionar que para hacer comparables los resultados, la fase de capa completamente conectada corresponde a la misma estructura de los tres modelos de perceptrón multicapa, sólo agregando las capas convolucionales y de pooling con combinaciones de las técnicas de regularización mencionadas.

In [20]:
model_cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation='softmax')
])

# Compilar el modelo
model_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model_cnn.summary()


c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [21]:
# Entrenar el modelo
model_cnn.fit(train_dataset, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 69s 449ms/step - accuracy: 0.6776 - loss: 2.6961
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 448ms/step - accuracy: 0.7903 - loss: 0.5448
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 449ms/step - accuracy: 0.8586 - loss: 0.3582
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 450ms/step - accuracy: 0.8820 - loss: 0.3035
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 448ms/step - accuracy: 0.8957 - loss: 0.2750
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 447ms/step - accuracy: 0.9112 - loss: 0.2318
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 448ms/step - accuracy: 0.9262 - loss: 0.1953
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 451ms/step - accuracy: 0.9284 - loss: 0.1779
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 67s 448ms/step - accuracy: 0.9456 - loss: 0.1562
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 447ms/step - accuracy: 0.9568 - loss: 0.1208


Se observa una disminución constante en los valores de pérdida y un aumento en el accuracy a través de las epochs, lo que indica que el modelo está aprendiendo a clasificar las imágenes y a generalizar para el conjunto de datos de entrenamiento.

In [22]:
# Evaluar el modelo
loss, accuracy = model_cnn.evaluate(test_dataset)
print(f"Pérdida Modelo 1 CNN: {loss}")
print(f"Exactitud Modelo 1 CNN: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.9108 - loss: 0.2837
Pérdida Modelo 1 CNN: 0.3695753514766693
Exactitud Modelo 1 CNN: 0.8758445978164673


Como resultado, el accuracy obtenido al evaluar el modelo es de 0.87, lo que indica que el modelo logra clasificar correctamente las imágenes de chihuahuas y muffins en un 87% de los casos, una mejora significativa en comparación con los modelos de perceptrón multicapa.

#### 4.2.2 Modelo 2: Regularización L2 y Dropout

In [23]:
# A continuación, se crea el modelo CNN con regularización L2 y Dropout en las capas densas.
model_cnn2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(len(class_names), activation='softmax')
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model_cnn2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_cnn2.summary()



c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [24]:
# Entrenar el modelo
model_cnn2.fit(train_dataset, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 68s 445ms/step - accuracy: 0.7352 - loss: 8.6867
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 443ms/step - accuracy: 0.8421 - loss: 3.7276
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 444ms/step - accuracy: 0.8543 - loss: 2.3625
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 443ms/step - accuracy: 0.8778 - loss: 1.7888
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 443ms/step - accuracy: 0.9019 - loss: 1.4468
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 66s 443ms/step - accuracy: 0.9083 - loss: 1.1928
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 403ms/step - accuracy: 0.9098 - loss: 1.0158
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 80s 392ms/step - accuracy: 0.9250 - loss: 0.8655
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 392ms/step - accuracy: 0.9286 - loss: 0.7521
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 378ms/step - accuracy: 0.9337 - loss: 0.7099


Al igual que en el modelo anterior, se observa una disminución constante en los valores de pérdida y un aumento en el accuracy a través de las epochs, lo que indica que el modelo está aprendiendo a clasificar las imágenes y a generalizar para el conjunto de datos de entrenamiento.

In [25]:
# Evaluar el modelo
loss, accuracy = model_cnn2.evaluate(test_dataset)

print(f"Pérdida Modelo 2 CNN: {loss}")
print(f"Exactitud Modelo 2 CNN: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.9236 - loss: 0.6870
Pérdida Modelo 2 CNN: 0.8070096373558044
Exactitud Modelo 2 CNN: 0.8445945978164673


Como resultado el accuracy obtenido al evaluar el modelo es de 0.84, lo que indica que el modelo logra clasificar correctamente las imágenes de chihuahuas y muffins en un 84% de los casos, manteniendo resultados altos junto al modelo anterior y mejorando significativamente en comparación con los modelos de perceptrón multicapa.

#### 4.2.3 Modelo 3: Regularización L2, Dropout y tasa de aprendizaje

A continuación, se crea el modelo CNN con regularización L2 y Dropout en las capas de convolución y densas. Además, al optimizador se le añade un decaimiento exponencial de la tasa de aprendizaje.

In [26]:
# Decaimiento de la tasa de aprendizaje
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=False
)
model_cnn3 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),  
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)
model_cnn3.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_cnn3.summary()



Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [27]:
# Entrenar el modelo
model_cnn3.fit(train_dataset, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 63s 410ms/step - accuracy: 0.6499 - loss: 48.8045
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 406ms/step - accuracy: 0.7861 - loss: 2.6625
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 402ms/step - accuracy: 0.7934 - loss: 1.9694
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 401ms/step - accuracy: 0.8098 - loss: 1.5451
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 398ms/step - accuracy: 0.8296 - loss: 1.2891
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.8406 - loss: 1.1255
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.8217 - loss: 1.0984
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 404ms/step - accuracy: 0.8528 - loss: 1.0069
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.8410 - loss: 0.9641
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 407ms/step - accuracy: 0.8509 - loss: 0.9769


Al igual que en ambos modelos de CNN, se observa una disminución constante en los valores de pérdida y un aumento en el accuracy a través de las epochs, lo que indica que el modelo está aprendiendo a clasificar las imágenes y a generalizar para el conjunto de datos de entrenamiento.

In [28]:
# Evaluar el modelo
loss, accuracy = model_cnn3.evaluate(test_dataset)
print(f"Pérdida Modelo 3 CNN: {loss}")
print(f"Exactitud Modelo 3 CNN: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - accuracy: 0.7321 - loss: 1.2047
Pérdida Modelo 3 CNN: 1.002831220626831
Exactitud Modelo 3 CNN: 0.8006756901741028


El accuracy obtenido al evaluar el modelo es de 0.80, lo que indica que el modelo logra clasificar correctamente las imágenes de chihuahuas y muffins en un 80% de los casos, manteniendo resultados altos junto a los modelos anteriores y mejorando significativamente en comparación con los modelos de perceptrón multicapa.

## 5. Comparación de resultados

| Modelo   | Accuracy FNN | Loss FNN | Accuracy CNN | Loss CNN |
|----------|--------------|----------|--------------|----------|
| Modelo 1                                          | 0.54         | 0.68     | 0.87         | 0.37     |
| Modelo 2 Dropout y regularización l2              | 0.45         | 1.22     | 0.84         | 0.80     |
| Modelo 3 Dropout, regularización y learning rate  | 0.54         | 0.93     | 0.80         | 1.00     |

Como se puede observar en la tabla comparativa, el modelo de redes neuronales convolucionales (CNN) obtiene mejores resultados en términos de accuracy y loss **en todos los modelos** en comparación con el modelo de perceptrón multicapa (FNN).

Por su parte, la pérdida en los FNN y en CNN es mayor en los modelos con modificaciones, lo que podría indicar que estos cambios pueden estar afectando la capacidad de aprendizaje del modelo.

En cuanto al tercer modelo con CNN se puede observar una disminución en el accuracy y un aumento en la pérdida, lo que podría indicar que la inserción de la técnica de decaimiento de la tasa de aprendizaje no está siendo efectiva en este caso.

Cabe destacar que en las ejecuciones se pudo observar que las FNN tomaron menos tiempo en ser entrenadas en comparación con las CNN, lo que podría deberse a la mayor complejidad de las CNN y a la cantidad de capas que estas poseen.



## 6. Conclusiones

En relación al desempeño de los modelos FNN, se observa que todos tienen un accuracy similar cercano al 50% en incluso por debajo en el modelo 2 (45%). Debido a que la cantidad de categorías a predecir son 2, un accuracy de 50% es equivalente a un modelo que predice al azar, lo que sugiere que el modelo no está aprendiendo patrones significativos en los datos de entrada.

Por su parte, el desempeño de los modelos CNN es significativamente mejor, con accuracies superiores al 80% en todos los casos y alcanzando un 87% para el primer modelo. Esto sugiere que las redes neuronales convolucionales son más efectivas para aprender patrones en imágenes y clasificarlas correctamente, en comparación con los perceptrones multicapa.

